In [1]:
import pandas as pd
import numpy as np
from gc import collect
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
import difflib

Funciones útiles

In [2]:
def change_names(df, inplace=False):
    return df.rename({column: column.lower() for column in df}, axis=1, inplace=inplace)


def fix_comuna(comuna: str) -> str:
    words = list(comuna.strip())
    for i in range(len(words)):
        word = words[i]
        if i == 0:
            words[i] = word.replace('?', 'Ñ')
        else:
            words[i] = word.replace('?', 'ñ')
    # print(words)
    return ''.join(words)

Importación de los datos y eliminación de columnas no importantes

In [3]:
excel = pd.ExcelFile('Base-de-Datos-de-Transporte-Público-Abril-2021.xlsx')
servicios_raw = pd.read_excel(excel, 'Servicios')
vehiculos_raw = pd.read_excel(excel, 'Vehículos')
recorridos_raw = pd.read_excel(excel, 'O-D Recorridos')
trazados_raw = pd.read_excel(excel, 'Trazados')
del excel
collect()
servicios_to_drop = ['NRO_LINEA', 'RUT_RESPONSABLE', 'NOMBRE_FANTASIA', 'TIPO_PERSONA', 'REGION.1', 'TELEFONO', 'FAX', 'EMAIL']
servicios_raw.drop(servicios_to_drop, axis=1, inplace=True)

In [5]:
chile = pd.read_csv("Latitud - Longitud Chile.csv")
to_drop = list(set(chile.columns) - {'Comuna', 'Región', 'Latitud (Decimal)', 'Longitud (decimal)'})
chile.drop(to_drop, axis=1, inplace=True)
regiones = {
    "\xa0Tarapacá": 1,
    "\xa0Metropolitana de Santiago": 13,
    "\xa0Valparaíso": 5,
    "\xa0Lib. Gral. Bernardo O' Higgins": 6,
    "\xa0Biobío": 8,      
    "\xa0La Araucanía": 9,
    "\xa0Maule": 7,
    "\xa0Los Lagos": 10,
    "\xa0Ñuble": 16,
    "\xa0Coquimbo": 4,
    "\xa0Los Ríos": 14,
    "\xa0Magallanes y Antártica Chilena": 12,
    "\xa0Aysén del Gral. C. Ibáñez del Campo": 11,
    "\xa0Antofagasta": 2,
    "\xa0Atacama": 3,
    "\xa0Arica y Parinacota": 15,
}
chile.replace({'Región': regiones}, inplace=True)
chile_dict = chile.to_dict(orient='split')
chile_dict = {comuna[0]: comuna[1:] for comuna in chile_dict['data']}
sorted(chile_dict)

['Algarrobo',
 'Alhué',
 'Alto Biobío',
 'Alto Hospicio',
 'Alto del Carmen',
 'Ancud',
 'Andacollo',
 'Angol',
 'Antofagasta',
 'Antuco',
 'Antártica',
 'Arauco',
 'Arica',
 'Aysén',
 'Buin',
 'Bulnes',
 'Cabildo',
 'Cabo de Hornos',
 'Cabrero',
 'Calama',
 'Calbuco',
 'Caldera',
 'Calera de Tango',
 'Calle Larga',
 'Camarones',
 'Camiña',
 'Canela',
 'Carahue',
 'Cartagena',
 'Casablanca',
 'Castro',
 'Catemu',
 'Cauquenes',
 'Cañete',
 'Cerrillos',
 'Cerro Navia',
 'Chaitén',
 'Chanco',
 'Chañaral',
 'Chiguayante',
 'Chile Chico',
 'Chillán',
 'Chillán Viejo',
 'Chimbarongo',
 'Cholchol',
 'Chonchi',
 'Chépica',
 'Cisnes',
 'Cobquecura',
 'Cochamó',
 'Cochrane',
 'Codegua',
 'Coelemu',
 'Coihueco',
 'Coinco',
 'Colbún',
 'Colchane',
 'Colina',
 'Collipulli',
 'Coltauco',
 'Combarbalá',
 'Concepción',
 'Conchalí',
 'Concón',
 'Constitución',
 'Contulmo',
 'Copiapó',
 'Coquimbo',
 'Coronel',
 'Corral',
 'Coyhaique',
 'Cunco',
 'Curacautín',
 'Curacaví',
 'Curaco de Vélez',
 'Curanilah

In [ ]:
lugar_sql[lugar_sql['comuna'] == 'Paiguano']
keys = list(chile.keys())
keys.sort()
keys

: 

Creación tabla "Servicio"

In [6]:
servicios_sql = servicios_raw.drop(['DOMICILIO', 'COMUNA'], axis=1)
change_names(servicios_sql, inplace=True)
servicios_sql.rename({'flota vigente': 'flota'}, axis=1, inplace=True)
orden_servicios = ['folio', 'region', 'tipo_servicio', 'flota', 'nombre_responsable']
servicios_sql = servicios_sql.loc[:, orden_servicios]
servicios_sql = servicios_sql.dropna().drop_duplicates(subset=['folio', 'region'])
servicios_sql

,folio,region,tipo_servicio,flota,nombre_responsable
0,10,1,TAXI COLECTIVO RURAL,1,FRANCISCO R. CARLOS MAMANI
1,4608,1,TAXI COLECTIVO RURAL,1,JORGE ANDRES SOTO LARA
2,27,1,TAXI COLECTIVO RURAL,1,RENE RUPERTO OYANADEL RIVEROS
3,67,1,TAXI COLECTIVO RURAL,1,JOSE LINO CUELLO YA?EZ
4,4917,1,TAXI COLECTIVO RURAL,1,LUIS PEDRO BOTTARELLI ARROYO
...,...,...,...,...,...
41520,6074,15,MINIBUS RURAL CORRIENTE,14,ASOCIACION GREMIAL PUERTO VALLE ARICA
41521,6111,15,MINIBUS RURAL CORRIENTE,15,ASOCIACIÓN GREMIAL DE TRANSPORTE URBANO RURAL ...
41522,6143,15,MINIBUS RURAL CORRIENTE,4,EMP DE TRANSP NACIONAL E INTERNACIONAL S AN LO...
41523,6155,15,MINIBUS RURAL CORRIENTE,10,EMPRESA DE TRANSPORTE LOGISTICO Y TURISMO VANF...


Creación tabla "Vehículo"

In [16]:
vehiculos_sql = change_names(vehiculos_raw)
vehiculos_sql.rename({'ppu': 'patente', 'fecha_ingreso_rnt': 'fecha_ingreso', 'ano_fabricacion': 'anho_fabricacion', 'region': 's_region', 'folio': 's_folio'}, axis=1, inplace=True)
orden_vehiculos = ['patente', 's_folio', 's_region', 'marca', 'fecha_ingreso', 'capacidad', 'anho_fabricacion', 'modelo', 'tipo_servicio']
vehiculos_sql = vehiculos_sql.loc[:, orden_vehiculos]
vehiculos_sql = vehiculos_sql.dropna().drop_duplicates(subset=['patente'])
vehiculos_sql['capacidad'] = vehiculos_sql['capacidad'].astype(int)
vehiculos_sql
# vehiculos_sql.duplicated(subset=['patente']).value_counts()

,patente,s_folio,s_region,marca,fecha_ingreso,capacidad,anho_fabricacion,modelo,tipo_servicio
0,RW2882,10,1,HYUNDAI,2007-06-26,4,2007,ACCENT GL 1.5,TAXI COLECTIVO RURAL
2,DWCY37,18,1,HYUNDAI,2012-05-15,4,2012,ACCENT RB GL 1.6,TAXI COLECTIVO RURAL
3,DGFX88,19,1,HYUNDAI,2011-08-30,4,2011,ACCENT RB GL 1.6,TAXI COLECTIVO RURAL
4,FPCP93,24,1,HYUNDAI,2013-05-03,4,2013,ACCENT RB GL 1.6,TAXI COLECTIVO RURAL
5,BCRG90,27,1,PEUGEOT,2016-06-07,4,2016,301 ALLURE HDI 1.6,TAXI COLECTIVO RURAL
...,...,...,...,...,...,...,...,...,...
136025,LWGR22,400031,14,MAXUS,2019-12-03,15,2020,NEW V80 FL VAN 2.5,MINIBUS URBANO CORRIENTE
136026,DGVT92,400031,14,JAC,2013-07-29,12,2011,REFINE 2.8,MINIBUS URBANO CORRIENTE
136027,JYTY93,400031,14,BRILLIANCE,2018-03-20,13,2018,H2L CRDI 2.8,MINIBUS URBANO CORRIENTE
136028,XG6630,400031,14,KIA MOTORS,2014-05-20,12,2004,BESTA ULTRA 3.0,MINIBUS URBANO CORRIENTE


Creación tabla "Lugar"

In [8]:
lugar_origen = recorridos_raw.loc[:, ['NOMBRE_ORIGEN', 'COMUNA_ORIGEN', 'DOMICILIO_ORIGEN']]
lugar_destino = recorridos_raw.loc[:, ['NOMBRE_DESTINO', 'COMUNA_DESTINO', 'DOMICILIO_DESTINO']]
lugar_origen.rename({'NOMBRE_ORIGEN': 'NOMBRE', 'COMUNA_ORIGEN': 'COMUNA', 'DOMICILIO_ORIGEN': 'DOMICILIO'}, axis=1, inplace=True)
lugar_destino.rename({'NOMBRE_DESTINO': 'NOMBRE', 'COMUNA_DESTINO': 'COMUNA', 'DOMICILIO_DESTINO': 'DOMICILIO'}, axis=1, inplace=True)
lugar_sql = pd.concat([lugar_origen, lugar_destino], axis=0).drop_duplicates(ignore_index=True).reset_index(drop=True)
change_names(lugar_sql, inplace=True)
lugar_sql.dropna(inplace=True)
lugar_sql.replace({'comuna': {'Calera': 'La Calera', 'Llaillay': 'Llay-Llay'}}, inplace=True)
lugar_sql['comuna'] = lugar_sql.comuna.apply(lambda x: fix_comuna(x))
lugar_sql['comuna'] = lugar_sql['comuna'].apply(lambda x: difflib.get_close_matches(x, chile['Comuna'])[0])
lugar_sql[['region', 'latitud', 'longitud']] = lugar_sql.comuna.apply(lambda x: pd.Series({'region': chile_dict[x][0], 'latitud': chile_dict[x][1], 'longitud': chile_dict[x][2]}))
lugar_sql = lugar_sql.astype({'region': 'int32'})
lugar_sql

,nombre,comuna,domicilio,region,latitud,longitud
0,IQUIQUE,Iquique,PEDRO PRADO 2382,1,-20.2439,-70.1389
1,TERMINAL BAJO MOLLE,Iquique,S/N,1,-20.2439,-70.1389
2,BAJO MOLLE,Iquique,BAJO MOLLE,1,-20.2439,-70.1389
3,IQUIQUE,Iquique,PEDRO PRADO 2360,1,-20.2439,-70.1389
4,SECTOR SUR,Iquique,S/N,1,-20.2439,-70.1389
...,...,...,...,...,...,...
12043,SAN GREGORIO,Ñiquén,ERNESTO ZU?IGA S/N,16,-36.2869,-71.8989
12044,SAN GREGORIO,Ñiquén,CALLE ESTADO COSTADO PONIENTE PLAZA DE ARMAS,16,-36.2869,-71.8989
12045,SAN GREGORIO,Ñiquén,CALLE ESTADO CONSTADO PONIENTE PLAZA DE ARMAS,16,-36.2869,-71.8989
12046,COIHUECO,Coihueco,COMERCIO ENTRE LUIS HERMOSILLA Y BALMACEDA (LA...,16,-36.6282,-71.8318


Creación tabla "Recorrido"

In [9]:
recorridos_sql = recorridos_raw.drop(['TIPO_SERVICIO', 'TIPO_TRAZADO'], axis=1)
change_names(recorridos_sql, inplace=True)

id_origen, id_destino = [], []
for nombre_origen, comuna_origen, domicilio_origen, nombre_destino, comuna_destino, domicilio_destino in recorridos_sql[['nombre_origen', 'comuna_origen', 'domicilio_origen', 'nombre_destino', 'comuna_destino', 'domicilio_destino']].to_numpy():
    id_origen.append(lugar_sql.index[(lugar_sql['nombre'] ==  nombre_origen) & (lugar_sql['comuna'] ==  comuna_origen) & (lugar_sql['domicilio'] == domicilio_origen)])
    id_destino.append(lugar_sql.index[(lugar_sql['nombre'] == nombre_destino) & (lugar_sql['comuna'] == comuna_destino) & (lugar_sql['domicilio'] == domicilio_destino)])

id_origen = [id[0] if not id.empty else np.nan for id in id_origen]
id_destino = [id[0] if not id.empty else np.nan for id in id_destino]
recorridos_sql["id_origen"] = np.array(id_origen)
recorridos_sql["id_origen"] = recorridos_sql["id_origen"].astype('Int64')
recorridos_sql["id_destino"] = np.array(id_destino)
recorridos_sql["id_destino"] = recorridos_sql["id_destino"].astype('Int64')
recorridos_sql.drop(['nombre_origen', 'comuna_origen', 'domicilio_origen', 'nombre_destino', 'comuna_destino', 'domicilio_destino'], axis=1, inplace=True)
recorridos_sql.rename({'folio': 's_folio', 'region': 's_region'}, axis=1, inplace=True)
recorridos_sql = recorridos_sql.dropna().drop_duplicates(subset=['s_folio', 's_region', 'nombre_recorrido'])
orden_recorridos = ['s_folio', 's_region', 'nombre_recorrido', 'id_origen', 'id_destino']
# recorridos_sql = recorridos_sql.dropna()
recorridos_sql

,s_region,s_folio,nombre_recorrido,id_origen,id_destino
0,1,800012,T,0,6398
1,1,200002,9,1,6399
2,1,200002,94,1,6399
3,1,500398,t,2,2
4,1,80048,T,2,2
...,...,...,...,...,...
11200,16,4947,T,6395,12040
11201,16,4947,V-1,6395,12041
11202,16,5984,T,6396,12042
11207,16,6,V-2,6359,12046


Creación tabla "Trazado"

In [10]:
trazados_sql = trazados_raw.loc[:, ['SENTIDO', 'CALLE', 'COMUNA']]
change_names(trazados_sql, inplace=True)
trazados_sql = trazados_sql.dropna().drop_duplicates()
trazados_sql

,sentido,calle,comuna
0,IDA,LUIS MIGUEL AMUNATEGUI,Iquique
1,IDA,SARGENTO ALDEA,Iquique
2,IDA,JUAN MARTINEZ,Iquique
3,IDA,MANUEL BULNES,Iquique
4,IDA,RUTA A 16,Iquique
...,...,...,...
326065,REGRESO,RUTA R90-P,Lumaco
326067,REGRESO,RUTA R42,Lumaco
326070,REGRESO,RUTA R60 P,Purén
326071,REGRESO,RUTA R86,Los Sauces


Creación tabla "Pertenece"

In [11]:
pertenece_sql = recorridos_raw.loc[:, ['FOLIO', 'REGION', 'NOMBRE_RECORRIDO']]
change_names(pertenece_sql, inplace=True)
pertenece_sql.rename({'folio': 's_folio', 'region': 's_region', 'nombre_recorrido': 'r_nombre_recorrido'}, axis=1, inplace=True)
pertenece_sql = pertenece_sql.dropna().drop_duplicates()
pertenece_sql

,s_folio,s_region,r_nombre_recorrido
0,800012,1,T
1,200002,1,9
2,200002,1,94
3,500398,1,t
4,80048,1,T
...,...,...,...
11204,5,16,V-3
11205,3440,16,R-1
11206,3440,16,R-1 - ENTRADA POR VICU?A MACKENNA
11207,6,16,V-2


Creación tabla "PasaPor"

In [12]:
pasapor_sql = trazados_raw.loc[:, ['FOLIO', 'REGION', 'NOMBRE_RECORRIDO', 'CALLE', 'COMUNA', 'SENTIDO', 'ORDEN']]
change_names(pasapor_sql, inplace=True)
pasapor_sql.rename({'folio': 's_folio', 'region': 's_region', 'nombre_recorrido': 'r_nombre_recorrido', 'calle': 't_calle', 'comuna': 't_comuna', 'sentido': 't_sentido'}, axis=1, inplace=True)
pasapor_sql = pasapor_sql.dropna().drop_duplicates(subset=list(set(pasapor_sql.columns) - {'orden'}))
rec_list = list(recorridos_sql.groupby(['s_folio', 's_region', 'nombre_recorrido']).groups)
filter = pasapor_sql[['s_folio', 's_region', 'r_nombre_recorrido']].apply(tuple, axis=1)
pasapor_sql = pasapor_sql[filter.isin(rec_list)]
pasapor_sql

,s_folio,s_region,r_nombre_recorrido,t_calle,t_comuna,t_sentido,orden
0,10,1,T,LUIS MIGUEL AMUNATEGUI,Iquique,IDA,1
1,10,1,T,SARGENTO ALDEA,Iquique,IDA,2
2,10,1,T,JUAN MARTINEZ,Iquique,IDA,3
3,10,1,T,MANUEL BULNES,Iquique,IDA,4
4,10,1,T,RUTA A 16,Iquique,IDA,5
...,...,...,...,...,...,...,...
326149,800025,16,R-4,EL CARMEN,El Carmen,REGRESO,26
326150,800025,16,R-4,RUTA N59Q,El Carmen,REGRESO,27
326151,800025,16,R-4,PEMUCO,Pemuco,REGRESO,28
326152,800025,16,R-4,RUTA N59Q,Pemuco,REGRESO,29


In [13]:
print(f"{len(servicios_sql) = }")
print(f"{len(vehiculos_sql) = }")
print(f"{len(lugar_sql) = }")
print(f"{len(pasapor_sql) = }")
# print(f"{len(pertenece_sql) = }")
print(f"{len(trazados_sql) = }")
print(f"{len(recorridos_sql) = }")
print(f"total = {len(servicios_sql)+len(vehiculos_sql)+len(lugar_sql)+len(pasapor_sql)+len(trazados_sql)+len(recorridos_sql)}")

len(servicios_sql) = 41525
len(vehiculos_sql) = 101446
len(lugar_sql) = 11111
len(pasapor_sql) = 222832
len(trazados_sql) = 69027
len(recorridos_sql) = 8761
total = 454702


In [17]:
servicios_sql.to_csv('servicio.csv', index=False)
vehiculos_sql.to_csv('vehiculo.csv', index=False)
lugar_sql.to_csv('lugar.csv', index=True, index_label='id')
pasapor_sql.to_csv('pasapor.csv', index=False)
# pertenece_sql.to_csv('pertenece.csv', index=False, sep=';')
trazados_sql.to_csv('trazado.csv', index=False)
recorridos_sql.to_csv('recorrido.csv', index=False)

Conexión a la base de datos

In [ ]:
HOST = "cc3201.dcc.uchile.cl"
USER = "cc3201"
PASS = "ambashoyomote("
PORT = 5510
DATABASE = "cc3201"

conn_string = f'postgresql+psycopg2://{USER}:{PASS}@{HOST}:{PORT}/{DATABASE}'
  
db = create_engine(conn_string)
conn = db.connect()

: 

In [ ]:
from sqlalchemy import text
session = Session(db)
for i in range(1, 17):
    query = f"""
        CREATE MATERIALIZED VIEW comunasregion{i} AS
        SELECT DISTINCT comuna FROM lugar
        WHERE region = {i};
    """
    session.execute(text(query))
session.commit()

: 

Dump a archivos csv

: 

Dump a la base de datos

In [ ]:
# servicios_sql.to_sql('servicio', con=conn, if_exists='append', index=False)
# vehiculos_sql.to_sql('vehiculo', con=conn, if_exists='append', index=False)
# recorridos_sql.to_sql('recorrido', con=conn, if_exists='append', index=False)
# lugar_sql.to_sql('lugar', con=conn, if_exists='append', index=True, index_label='id')
# pertenece_sql.to_sql('pertenece', con=conn, if_exists='append', index=False)
# trazados_sql.to_sql('trazado', con=conn, if_exists='append', index=False)
# pasapor_sql.to_sql('pasapor', con=conn, if_exists='append', index=False)
conn.close()

: 

In [ ]:
conn.close()

: 